In [0]:
from datetime import datetime
from datetime import timedelta

In [0]:
# Function to identify if a table already exists
def table_exists(schema_name, database_name, table_name):
    spark.catalog.setCurrentCatalog(schema_name)
    
    return spark.catalog.tableExists(f'{database_name}.{table_name}')


In [0]:
spark.catalog.setCurrentCatalog('gold')

if spark.catalog.tableExists('nytimes.weekly_trending_persons'):
    last_updated = spark.sql('SHOW PARTITIONS gold.nytimes.weekly_trending_persons').orderBy('ref_date', ascending = False).limit(1).collect()[0][0]

    end_date = spark.sql('SHOW PARTITIONS silver.nytimes.top_stories').orderBy('ref_date', ascending = False).limit(1).collect()[0][0]

    if last_updated == end_date:
        print('Table is already up to date')
        dbutils.notebook.exit()

    start_date = spark.sql(f'''
        SELECT 
            MIN(ref_date) 
        FROM silver.nytimes.top_stories 
        WHERE weekofyear(ref_date) = {datetime.strptime(end_date, "%Y-%m-%d").date().strftime('%U')}
    ''').collect()[0][0]
    
    print(f'Updating table with data from {start_date} to {end_date}')
else:
    start_date = spark.sql('SHOW PARTITIONS silver.nytimes.top_stories').orderBy('ref_date').limit(1).collect()[0][0]
    end_date = spark.sql('SHOW PARTITIONS silver.nytimes.top_stories').orderBy('ref_date', ascending = False).limit(1).collect()[0][0]
    print(f'Creating table with data from {start_date} to {end_date}') 


In [0]:
df_upload = spark.sql(f'''
  WITH persons AS (
    SELECT
      ref_date,
      weekofyear(ref_date) AS week_number,
      explode(ds_persons) AS person
    FROM silver.nytimes.top_stories
    WHERE ref_date >= '{start_date}'
  ),

  week_period AS (
    SELECT
      week_number,
      MIN(ref_date) AS week_start,
      MAX(ref_date) AS week_end
    FROM persons
    GROUP BY week_number
  )

  SELECT
    p.week_number,
    week_start,
    week_end,
    concat(
      split_part(regexp_replace(person, r'\\s\\(\\d\\d\\d\\d\\-?.?.?.?.', ''), ',', '2'),
      ' ',
      split_part(regexp_replace(person, r'\\s\\(\\d\\d\\d\\d\\-?.?.?.?.', ''), ',', '1')
    ) AS person,
    COUNT(*) AS mentions,
    week_end AS ref_date
  FROM persons p
  INNER JOIN week_period ON p.week_number = week_period.week_number
  GROUP BY p.week_number, week_start, week_end, person
  ORDER BY week_number DESC, mentions DESC
'''.format(start_date = start_date))

In [0]:
(df_upload.write
    .partitionBy('ref_date')
    .format("delta")
    .mode('overwrite')
    .option('replaceWhere', f'ref_date >= "{start_date}"')
    .saveAsTable("gold.nytimes.weekly_trending_persons")
)

In [0]:
%sql

SHOW PARTITIONS gold.nytimes.weekly_trending_persons;

In [0]:
%sql

SELECT * FROM  gold.nytimes.weekly_trending_persons;